In [53]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys


import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense
from keras.optimizers import SGD, Adam, rmsprop

%matplotlib inline

In [54]:
print(os.listdir("../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen"))

['AKIEC', 'BKL', 'NV', 'MEL', 'VASC', 'DF', 'BCC']


In [55]:
resnet50_weights = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [56]:
space = {
         'lr': hp.choice('lr',[0.01, 0.001, 0.0001]),
#          'dropout': hp.choice('dropout', [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]),
         'batch_size': hp.choice('batch_size', [32, 64, 128]),
         'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#        'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'activation': hp.choice('activation',['relu',
#                                                 'tanh']),
        }

In [57]:
train_dir = '../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen'
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen 0
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/AKIEC 327
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/BKL 1099
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/NV 6705
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/MEL 1113
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/VASC 142
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/DF 115
../input/skin-classification-preprocess-sharpen/train-preprocess-sharpen/train-preprocess-sharpen/BCC 514


In [58]:
def f_nn(params):   
    print ('Parameters testing: ', params)
    batch_size=params['batch_size']
    epochs=params['epochs']
    from keras.preprocessing.image import ImageDataGenerator

    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            fill_mode='nearest',
        horizontal_flip=True,
        validation_split=0.25)

    # target_size = (height, width)
    target_size = (450, 600)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,       
            class_mode = 'categorical',
            batch_size=batch_size,
            subset="training",
            shuffle = True)

    validation_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,        
            class_mode = 'categorical',
            batch_size = batch_size,
            subset = "validation",
            shuffle = True)



    
    model = Sequential()
    model.add(ResNet50(include_top=False, pooling='avg', weights=resnet50_weights))
    model.add(Dense(output_classes, activation='softmax'))
    model.layers[0].trainable = False
    model.compile(optimizer = Adam(lr=params['lr']), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = 400,
      epochs = epochs,
      validation_data = validation_generator,
      validation_steps = 100,
      verbose = 1, callbacks=get_callbacks(params))
    
    
    best_epoch = np.argmax(history.history['val_acc'])
    best_val_acc = np.max(history.history['val_acc'])
    print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'val_acc': best_val_acc, 'best_epoch': best_epoch, 'eval_time': time.time(), 'status': STATUS_OK}

In [59]:
# os.mkdir('callbacks')

In [60]:
# os.mkdir('tensor-logs')

In [61]:
 # !ls

In [62]:
output_classes = 7
input_shape = (450,600,3)

In [ ]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
                ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
             TensorBoard('tensor-logs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
    return callbacks

____

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'batch_size': 64, 'epochs': 25, 'lr': 0.01}
Found 7515 images belonging to 7 classes.
Found 2500 images belonging to 7 classes.
